<a href="https://colab.research.google.com/github/pavstar619/Churn-Prediction-Telco/blob/master/Churn_Prediction_Telco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
#Dataset
df = pd.read_csv('https://raw.githubusercontent.com/pavstar619/Churn-Prediction-Telco/master/telco.csv')
df.shape


(7043, 21)